In [141]:
!pip install nltk
!pip install spacy
!python -m spacy download pt_core_news_sm

2023-08-19 21:25:48.948120: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 21:25:55.860972: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 30.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [142]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

import string

In [165]:
df = pd.read_csv('./olist.csv')
df2 = pd.read_csv('./TweetsWithTheme.csv')
df = df.drop(["review_text", "review_text_tokenized", "polarity", "kfold_rating", "kfold_polarity", "original_index"], axis=1)
df2 = df2.drop(["id", "tweet_date", "query_used"], axis=1)
df = df[:24000]
df2 = df2[:8000]
df.head()

,review_text_processed,rating
0,perfeito....chegou antes do prazo.....,5
1,foi uma otima compra! chegou antes mesmo do pr...,5
2,recebi muito rapido e um otimo custo beneficio,5
3,recomendo,5
4,so veio uma capa comprei 3 ai paguei. mais de ...,1


In [144]:
df2

,tweet_text,sentiment
0,ela vendo a família :( FUGIDINHA VERA LÚCIA,Negativo
1,Eu odeio o fato que muito provavelmente meu vo...,Negativo
2,"Ciro ""mita"" tanto quanto aquele outro, não que...",Negativo
3,Raiva do Bolsonaro agora :),Positivo
4,Reflexos da alienação petista comandada por Lu...,Negativo
...,...,...
7995,@ThiagoGroening Lula na cadeia :D,Positivo
7996,@GorilaQuevoa ele é candidato a deputado feder...,Positivo
7997,Estava sem saber em quem votar até conhecer as...,Positivo
7998,Preciso contar uma coisa pros meus amigos do f...,Positivo


In [166]:
df["rating"] = np.where(df["rating"] < 3, 0, 1)
df2["sentiment"] = np.where(df2["sentiment"] == "Positivo", 1, 0)

In [167]:
df = df.dropna()
df2 = df2.dropna()

In [168]:
personal_text = ["Local ruim e feio, cheira mal", "local ruim, feio e sujo", "Esse lugar é um verdadeiro desastre - feio, sujo e com um cheiro insuportável.", "Nunca vi um local tão horrível em toda a minha vida - feio, malcuidado e fedorento.", "Esse é o tipo de lugar que você entra e já sente vontade de sair correndo - é feio, sujo e o cheiro é simplesmente repugnante.", "Não há como ignorar o fato de que esse local é uma combinação terrível de feiura, sujeira e mau odor.", "É impressionante como um lugar pode ser tão ruim em todos os sentidos - feio, sujo e com um cheiro que parece não ter fim.", "Não dá para suportar ficar aqui por muito tempo - é um lugar feio, sujo e com um fedor que gruda na pele.", "Parece que todas as coisas ruins se reuniram nesse lugar - é feio, sujo e o cheiro é simplesmente insuportável.", "Não entendo como alguém pode frequentar esse local - é feio, sujo e o cheiro deixa qualquer um enjoado.", "Esse lugar parece ter sido esquecido pelo resto do mundo - é feio, sujo e o cheiro é quase inacreditável.", "Não há como dar um passo sequer aqui sem ser atingido pela combinação de feiura, sujeira e um odor nauseante."]
personal_sentiment = [0 for i in range(len(personal_text))]

In [169]:
texts = personal_text + list(df["review_text_processed"].values) + list(df2["tweet_text"].values)
resp = personal_sentiment + list(df["rating"].values) + list(df2["sentiment"].values)

df = pd.DataFrame({
    "review_text_processed": texts,
    "rating": resp
})

In [149]:
df.shape

(33012, 2)

In [150]:
df.isnull().sum()

review_text_processed    0
rating                   0
dtype: int64

In [151]:
df = df.dropna()

In [152]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [153]:
stop_words = nltk.corpus.stopwords.words('portuguese')

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33012 entries, 0 to 33011
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   review_text_processed  33012 non-null  object
 1   rating                 33012 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 515.9+ KB


In [170]:
tweets = df["review_text_processed"]
tokenization = [word_tokenize(text.lower()) for text in tweets]

In [171]:
lemmatizer = WordNetLemmatizer()

new_tweets = []

for phrase in tokenization:
  new_phrase = ""
  for token in phrase:
    if not str(token) in stop_words and not token in string.punctuation and "@" not in token and "http" not in token and len(token) > 1 and not token.isdigit():
      new_phrase += lemmatizer.lemmatize(str(token)) + " "
  new_tweets.append(new_phrase[:-1])

In [172]:
df["review_text_processed"] = new_tweets

In [173]:
vect_uni_idf = TfidfVectorizer(ngram_range=(1,1), use_idf=True, norm='l2', stop_words=stop_words)
text_vect_uni_idf = vect_uni_idf.fit_transform(df["review_text_processed"])

X_trainUIDF, X_testUIDF, y_trainUIDF, y_testUIDF = train_test_split(text_vect_uni_idf, df["rating"], test_size=0.2, random_state=42)

In [174]:
rfcidf = RandomForestClassifier()

rfcidf.fit(X_trainUIDF, y_trainUIDF)
y_predUIDF = rfcidf.predict(X_testUIDF)

In [175]:
print(accuracy_score(y_testUIDF, y_predUIDF))

0.8477276276745276


In [176]:
rfcidf.predict(vect_uni_idf.transform(["local ruim, feio e sujo com cheiro de peixe"]))

array([0])

In [162]:
dtc = DecisionTreeClassifier()

dtc.fit(X_trainUIDF, y_trainUIDF)
y_preddtc = dtc.predict(X_testUIDF)

print(accuracy_score(y_testUIDF, y_preddtc))

0.8090262002120249


In [179]:
import joblib
model_filename = 'sentiment_analyser_model.pkl'
joblib.dump(rfcidf, model_filename)

['sentiment_analyser_model.pkl']

In [180]:
model_filename = 'sentiment_analyser_vect.pkl'
joblib.dump(vect_uni_idf, model_filename)

['sentiment_analyser_vect.pkl']